# Case Study: House Price Regression

This case study uses **house_price_regression_dataset.csv** and investigates:
- **Studentization of coefficients** (reliability / statistical significance via OLS t-statistics),
- **Standardization of coefficients** (effect size / relative feature influence),
- and **Regularization methods** (**Ridge**, **Lasso**, **Elastic-Net**) for controlling overfitting and improving generalization.


Key setup:
- **Random 70% / 30% Train/Test split**
- Two data variants:
  - **Unscaled:** raw $X$, raw $y$
  - **Scaled:** **both** $X$ and $y$ standardized using **training-set statistics only**
- Metrics reported on **both Train and Test**:
  - **MSE**, **MAE**, **R²**  
- For **Scaled models**, metrics are computed in **original y-units** by inverse-transforming predictions (so they are directly comparable with Unscaled).

---

## Five parts
1. Load dataset, split, create **scaled & unscaled** sets. Train/test evaluation for **Linear, Ridge, Lasso, ElasticNet** on both variants.
2. **Linear Regression:** additionally compute **OLS t-statistics (studentized coefficients)** for both variants (unscaled + scaled). Also show **standardized coefficients**.
3. **Ridge:** $\lambda\in \log(1,1000)$ with **10 points**. Sort results **best → worst** by Test MSE. Show standardized coefficients for the **best** model (scaled & unscaled).
4. **Lasso:** same as (3).
5. **ElasticNet:** same as (3). (ElasticNet uses `l1_ratio = 0.5`.)

> Note: In the **Scaled** setting (both $X$ and $y$ scaled), the fitted coefficients are already **standardized effect sizes**.


In [1]:
# === Imports ===
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet

import statsmodels.api as sm

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 140)


In [2]:
# === Load dataset ===
DATA_PATH = "house_price_regression_dataset.csv"
df = pd.read_csv(DATA_PATH)

print("Shape:", df.shape)
display(df.head())

target_col = df.columns[-1]  # House_Price
print("Target column:", target_col)


Shape: (1000, 8)


Square_Footage  Num_Bedrooms  Num_Bathrooms  Year_Built  Lot_Size  Garage_Size  Neighborhood_Quality   House_Price
0            1360             2              1        1981  0.599637            0                     5  2.623829e+05
1            4272             3              3        2016  4.753014            1                     6  9.852609e+05
2            3592             1              2        2016  3.634823            0                     9  7.779774e+05
3             966             1              2        1977  2.730667            1                     8  2.296989e+05
4            4926             2              1        1993  4.699073            0                     8  1.041741e+06

Target column: House_Price


In [4]:
# === Train/Test split (70/30) ===
X = df.drop(columns=[target_col]).copy()
y = df[target_col].values.reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42, shuffle=True
)

print("Train:", X_train.shape, y_train.shape)
print("Test :", X_test.shape, y_test.shape)


Train: (700, 7) (700, 1)
Test : (300, 7) (300, 1)


In [5]:
# === Create unscaled and scaled versions (scale BOTH X and y) ===
# Unscaled
X_train_un = X_train.values
X_test_un  = X_test.values
y_train_un = y_train
y_test_un  = y_test

# Scaled (fit scalers only on TRAIN)
x_scaler = StandardScaler()
y_scaler = StandardScaler()

X_train_sc = x_scaler.fit_transform(X_train_un)
X_test_sc  = x_scaler.transform(X_test_un)

y_train_sc = y_scaler.fit_transform(y_train_un)
y_test_sc  = y_scaler.transform(y_test_un)

feature_names = X.columns.tolist()

print("Feature names:", feature_names)
print("Scaled sanity check:")
print("X_train_sc mean/std (first 3):", np.mean(X_train_sc, axis=0)[:3], np.std(X_train_sc, axis=0)[:3])
print("y_train_sc mean/std:", float(np.mean(y_train_sc)), float(np.std(y_train_sc)))


Feature names: ['Square_Footage', 'Num_Bedrooms', 'Num_Bathrooms', 'Year_Built', 'Lot_Size', 'Garage_Size', 'Neighborhood_Quality']
Scaled sanity check:
X_train_sc mean/std (first 3): [1.81442163e-16 7.35919262e-17 8.12048841e-17] [1. 1. 1.]
y_train_sc mean/std: -1.586032892321652e-17 0.9999999999999999


## Helper functions

In [6]:
def eval_metrics(y_true, y_pred):
    y_true = y_true.reshape(-1)
    y_pred = y_pred.reshape(-1)
    return {
        "mse": mean_squared_error(y_true, y_pred),
        "mae": mean_absolute_error(y_true, y_pred),
        "r2":  r2_score(y_true, y_pred),
    }

def standardized_coef_unscaled(beta, X_train_raw, y_train_raw):
    """Standardized coefficients (effect size) for unscaled data:
       beta_std = beta * (std(X)/std(y)), computed using TRAIN statistics.
    """
    std_x = np.std(X_train_raw, axis=0, ddof=0)
    std_y = float(np.std(y_train_raw, axis=0, ddof=0))
    std_y = std_y if std_y > 0 else 1.0
    return beta * (std_x / std_y)

def convert_scaled_coef_to_original(beta_scaled, intercept_scaled, x_scaler: StandardScaler, y_scaler: StandardScaler):
    """Convert coefficients from (X_scaled, y_scaled) space to original units."""
    x_mean = x_scaler.mean_
    x_std  = x_scaler.scale_
    y_mean = float(y_scaler.mean_)
    y_std  = float(y_scaler.scale_)

    beta_orig = (y_std * beta_scaled) / x_std
    intercept_orig = y_mean + y_std * intercept_scaled - np.sum((y_std * beta_scaled * x_mean) / x_std)
    return beta_orig, intercept_orig


# Part 1 — Baseline evaluation (Linear, Ridge, Lasso, ElasticNet) on unscaled & scaled

In this part, we establish a **baseline comparison** between four common regression models using **both unscaled and scaled versions** of the dataset.

✅ **Important note (default settings):**  
All models in this section are trained using their **default hyperparameter values** in scikit-learn:

- **Linear Regression:** no regularization  
- **Ridge Regression:** default regularization strength $\alpha = 1.0$  
- **Lasso Regression:** default regularization strength $\alpha = 1.0$  
- **Elastic-Net Regression:** default $\alpha = 1.0$ and default mixture ratio $l1\_ratio = 0.5$

We evaluate each model on both the **training set** and the **test set** using:
- Mean Squared Error (MSE)
- Mean Absolute Error (MAE)
- $R^2$ score

---

## Data representation

Let the dataset contain $n$ samples and $d$ input features.  
We define the design matrix as:

$$
X \in \mathbb{R}^{n \times (d+1)}
$$

where the **first column of $X$ is a bias column of ones**, i.e.,

$$
X = \begin{bmatrix}
1 & x_{11} & x_{12} & \dots & x_{1d}\\
1 & x_{21} & x_{22} & \dots & x_{2d}\\
\vdots & \vdots & \vdots & \ddots & \vdots\\
1 & x_{n1} & x_{n2} & \dots & x_{nd}
\end{bmatrix}
$$

The model parameter vector is:

$$
\beta \in \mathbb{R}^{(d+1)}
$$

and the regression model is expressed compactly as:

$$
\hat{y} = X\beta
$$

---

## Models and objective functions

### 1) Linear Regression (Ordinary Least Squares)
Linear regression learns $\beta$ by minimizing the squared error:

$$
\min_{\beta}\; \sum_{i=1}^{n} (y_i - \hat{y}_i)^2
\quad=\quad
\min_{\beta}\; \|y - X\beta\|_2^2
$$

---

### 2) Ridge Regression ($\ell_2$ regularization)
Ridge regression adds an $\ell_2$ penalty on coefficients:

$$
\min_{\beta}\; \|y - X\beta\|_2^2 + \alpha \|\beta\|_2^2
$$

where:
$$
\|\beta\|_2^2 = \sum_{j=1}^{d+1} \beta_j^2
$$

---

### 3) Lasso Regression ($\ell_1$ regularization)
Lasso regression adds an $\ell_1$ penalty, encouraging sparse coefficients:

$$
\min_{\beta}\; \|y - X\beta\|_2^2 + \alpha \|\beta\|_1
$$

where:
$$
\|\beta\|_1 = \sum_{j=1}^{d+1} |\beta_j|
$$

---

### 4) Elastic-Net Regression ($\ell_1 + \ell_2$ regularization)
Elastic-Net combines Ridge and Lasso penalties:

$$
\min_{\beta}\; \|y - X\beta\|_2^2
+ \alpha\Big( r\|\beta\|_1 + (1-r)\|\beta\|_2^2 \Big)
$$

where:
- $\alpha$ controls the overall regularization strength
- $r$ is the mixing parameter (in scikit-learn: `l1_ratio`)

In this notebook, we use the default:
$$
r = 0.5
$$

---

✅ **Goal of Part 1:**  
This baseline provides a reference point for understanding model behavior **before tuning $\alpha$ (or $\lambda$)**, and supports the deeper analysis in later parts on coefficient reliability, effect size, and regularization trends.


In [7]:
def fit_and_report_basic_models():
    models = {
        "Linear": LinearRegression(),
        "Ridge(alpha=1.0)": Ridge(alpha=1.0, random_state=42),
        "Lasso(alpha=1.0)": Lasso(alpha=1.0, max_iter=50000, random_state=42),
        "ElasticNet(alpha=1.0,l1_ratio=0.5)": ElasticNet(alpha=1.0, l1_ratio=0.5, max_iter=50000, random_state=42),
    }

    rows = []
    for variant in ["unscaled", "scaled"]:
        if variant == "unscaled":
            Xt_tr, Xt_te = X_train_un, X_test_un
            yt_tr = y_train_un
        else:
            Xt_tr, Xt_te = X_train_sc, X_test_sc
            yt_tr = y_train_sc

        for name, model in models.items():
            model.fit(Xt_tr, yt_tr.ravel())
            pred_tr = model.predict(Xt_tr).reshape(-1,1)
            pred_te = model.predict(Xt_te).reshape(-1,1)

            # Evaluate in original y-units for direct comparability
            if variant == "scaled":
                pred_tr_eval = y_scaler.inverse_transform(pred_tr)
                pred_te_eval = y_scaler.inverse_transform(pred_te)
                yt_tr_eval = y_train_un
                yt_te_eval = y_test_un
            else:
                pred_tr_eval = pred_tr
                pred_te_eval = pred_te
                yt_tr_eval = y_train_un
                yt_te_eval = y_test_un

            mtr = eval_metrics(yt_tr_eval, pred_tr_eval)
            mte = eval_metrics(yt_te_eval, pred_te_eval)

            rows.append({
                "variant": variant,
                "model": name,
                **{f"train_{k}": v for k,v in mtr.items()},
                **{f"test_{k}": v for k,v in mte.items()},
            })
    return pd.DataFrame(rows)

part1_results = fit_and_report_basic_models()
display(part1_results.sort_values(["variant","test_mse"]))


variant                               model     train_mse      train_mae  train_r2      test_mse       test_mae   test_r2
4    scaled                              Linear  9.277129e+07    7614.585671  0.998559  1.022811e+08    8089.577892  0.998398
5    scaled                    Ridge(alpha=1.0)  9.290165e+07    7615.504206  0.998557  1.035727e+08    8161.347926  0.998378
7    scaled  ElasticNet(alpha=1.0,l1_ratio=0.5)  2.950095e+10  146733.399291  0.541891  2.901786e+10  146283.933695  0.545492
6    scaled                    Lasso(alpha=1.0)  6.439721e+10  217690.134646  0.000000  6.384958e+10  218131.524136 -0.000079
1  unscaled                    Ridge(alpha=1.0)  9.277193e+07    7615.078712  0.998559  1.022768e+08    8090.285780  0.998398
2  unscaled                    Lasso(alpha=1.0)  9.277130e+07    7614.625698  0.998559  1.022799e+08    8089.570891  0.998398
0  unscaled                              Linear  9.277129e+07    7614.585671  0.998559  1.022811e+08    8089.577892  0.998398
3  unscaled  ElasticNet(alpha=1.0,l1_ratio=0.5)  1.319714e+08    9216.971478  0.997951  1.429595e+08    9649.067608  0.997761

# Part 2 — Linear Regression + OLS studentized (t-statistics)

In this part, we focus on **Linear Regression** and examine the learned coefficients from **two complementary perspectives**:

1. **Reliability (Studentization):**  
   We use **Ordinary Least Squares (OLS)** to compute **studentized coefficients**, i.e., **t-statistics**, which quantify whether each coefficient is statistically distinguishable from zero (under standard OLS assumptions).  
   This provides a measure of *how reliable* each coefficient estimate is.

2. **Effect size (Standardization):**  
   We compute **standardized coefficients**, which quantify the *relative influence* of each feature after accounting for differences in feature scale.  
   This provides a measure of *how strong* the effect of each feature is on the output.

---

## What is trained and compared?

We train **Linear Regression** in two settings:

### 1) Unscaled setting
- Train using raw features $X$ and raw target $y$  
- Report:
  - Train/Test metrics: **MSE, MAE, $R^2$**
  - **Standardized coefficients** (effect size)
  - **OLS t-statistics (studentized coefficients)** and p-values (reliability)

### 2) Scaled setting
- Train using standardized features and standardized target:
  - $X_{scaled}$: standardized using training-set mean and standard deviation  
  - $y_{scaled}$: standardized using training-set mean and standard deviation  
- Report:
  - Train/Test metrics computed in **original units of $y$** by inverse-transforming predictions
  - Coefficients in scaled space (already interpretable as standardized effect sizes)
  - **OLS t-statistics (studentized coefficients)** and p-values computed in the scaled space

---

## Studentization (reliability): OLS t-statistics

Using the OLS model:

$$
\hat{y} = X\hat{\beta}
$$

the **studentized coefficient** (t-statistic) for feature $j$ is:

$$
t_j = \frac{\hat{\beta}_j}{SE(\hat{\beta}_j)}
$$

A larger magnitude $|t_j|$ indicates higher confidence that feature $j$ has a non-zero contribution.

---

## Standardization (effect): standardized coefficients

Standardized coefficients measure effect size relative to feature and output variability.

### For the unscaled case
We compute:

$$
\beta_j^{std} = \hat{\beta}_j \cdot \frac{\sigma_{x_j}}{\sigma_y}
$$

where $\sigma_{x_j}$ is the standard deviation of feature $j$ (computed on training data) and $\sigma_y$ is the standard deviation of the target (training data).

### For the scaled case
Since both $X$ and $y$ are standardized, the fitted coefficients already correspond to standardized effects:
- a 1 standard deviation change in feature $x_j$ produces approximately $\beta_j$ standard deviations change in $y$

---

✅ **Goal of Part 2:**  
To clearly separate *feature importance* into:
- **Effect size** (standardized coefficients)  
- **Reliability** (studentized coefficients via OLS t-statistics)


In [8]:
def linear_with_ols_reports():
    reports = {}
    for variant in ["unscaled", "scaled"]:
        if variant == "unscaled":
            Xt_tr, Xt_te = X_train_un, X_test_un
            yt_tr = y_train_un
        else:
            Xt_tr, Xt_te = X_train_sc, X_test_sc
            yt_tr = y_train_sc

        lr = LinearRegression()
        lr.fit(Xt_tr, yt_tr.ravel())

        pred_tr = lr.predict(Xt_tr).reshape(-1,1)
        pred_te = lr.predict(Xt_te).reshape(-1,1)

        # Evaluate in original y-units
        if variant == "scaled":
            pred_tr_eval = y_scaler.inverse_transform(pred_tr)
            pred_te_eval = y_scaler.inverse_transform(pred_te)
        else:
            pred_tr_eval = pred_tr
            pred_te_eval = pred_te

        mtr = eval_metrics(y_train_un, pred_tr_eval)
        mte = eval_metrics(y_test_un, pred_te_eval)

        # OLS (t-stats)
        X_sm = sm.add_constant(Xt_tr)
        ols = sm.OLS(yt_tr, X_sm).fit()

        feat_names = ["const"] + feature_names
        ols_table = pd.DataFrame({
            "term": feat_names,
            "beta_hat": ols.params.reshape(-1),
            "t_value": ols.tvalues.reshape(-1),
            "p_value": ols.pvalues.reshape(-1)
        }).sort_values("t_value", key=np.abs, ascending=False)

        # Standardized coefficients
        beta = lr.coef_.reshape(-1)
        if variant == "unscaled":
            beta_std = standardized_coef_unscaled(beta, X_train_un, y_train_un)
            coef_table = pd.DataFrame({
                "feature": feature_names,
                "coef_value": beta,
                "standardized_coef": beta_std
            }).sort_values("standardized_coef", key=np.abs, ascending=False)
        else:
            # scaled X and scaled y => beta already standardized effect size
            beta_orig, intercept_orig = convert_scaled_coef_to_original(beta, float(lr.intercept_), x_scaler, y_scaler)
            coef_table = pd.DataFrame({
                "feature": feature_names,
                "coef_value_(scaled_space)=standardized": beta,
                "coef_value_(original_units)": beta_orig
            }).sort_values("coef_value_(scaled_space)=standardized", key=np.abs, ascending=False)

        reports[variant] = {
            "train_metrics": mtr,
            "test_metrics": mte,
            "coef_table": coef_table,
            "ols_table": ols_table
        }
    return reports

lin_reports = linear_with_ols_reports()

for variant in ["unscaled","scaled"]:
    print(f"\n{'='*90}")
    print(f"LINEAR REGRESSION REPORT — {variant.upper()}")
    print(f"{'-'*90}")
    print("Train metrics:", {k: round(v, 6) for k,v in lin_reports[variant]['train_metrics'].items()})
    print("Test  metrics:", {k: round(v, 6) for k,v in lin_reports[variant]['test_metrics'].items()})

    print("\nStandardized coefficients:")
    display(lin_reports[variant]["coef_table"])

    print("\nOLS t-statistics (studentized coefficients):")
    display(lin_reports[variant]["ols_table"])



LINEAR REGRESSION REPORT — UNSCALED
------------------------------------------------------------------------------------------
Train metrics: {'mse': 92771292.929895, 'mae': 7614.585671, 'r2': 0.998559}
Test  metrics: {'mse': 102281056.957654, 'mae': 8089.577892, 'r2': 0.998398}

Standardized coefficients:


/tmp/ipython-input-3445319319.py:15: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  std_y = float(np.std(y_train_raw, axis=0, ddof=0))
/tmp/ipython-input-3445319319.py:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_mean = float(y_scaler.mean_)
/tmp/ipython-input-3445319319.py:24: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_std  = float(y_scaler.scale_)


feature    coef_value  standardized_coef
0        Square_Footage    199.353671           0.984754
3            Year_Built   1002.747913           0.081853
4              Lot_Size  14844.903526           0.075966
1          Num_Bedrooms  10238.036885           0.056994
2         Num_Bathrooms   8312.140025           0.026746
5           Garage_Size   5202.225589           0.016866
6  Neighborhood_Quality    132.238378           0.001514


OLS t-statistics (studentized coefficients):


term      beta_hat     t_value        p_value
1        Square_Footage  1.993537e+02  675.234289   0.000000e+00
0                 const -2.028686e+06  -57.482227  1.070092e-265
4            Year_Built  1.002748e+03   56.635933  5.089432e-262
5              Lot_Size  1.484490e+04   52.179622  4.644083e-242
2          Num_Bedrooms  1.023804e+04   39.098976  2.247565e-177
3         Num_Bathrooms  8.312140e+03   18.470720   3.123975e-62
6           Garage_Size  5.202226e+03   11.572958   1.937848e-28
7  Neighborhood_Quality  1.322384e+02    1.045980   2.959354e-01


LINEAR REGRESSION REPORT — SCALED
------------------------------------------------------------------------------------------
Train metrics: {'mse': 92771292.929895, 'mae': 7614.585671, 'r2': 0.998559}
Test  metrics: {'mse': 102281056.95765, 'mae': 8089.577892, 'r2': 0.998398}

Standardized coefficients:


feature  coef_value_(scaled_space)=standardized  coef_value_(original_units)
0        Square_Footage                                0.984754                   199.353671
3            Year_Built                                0.081853                  1002.747913
4              Lot_Size                                0.075966                 14844.903526
1          Num_Bedrooms                                0.056994                 10238.036885
2         Num_Bathrooms                                0.026746                  8312.140025
5           Garage_Size                                0.016866                  5202.225589
6  Neighborhood_Quality                                0.001514                   132.238378


OLS t-statistics (studentized coefficients):


term  beta_hat     t_value        p_value
1        Square_Footage  0.984754  675.234289   0.000000e+00
4            Year_Built  0.081853   56.635933  5.089432e-262
5              Lot_Size  0.075966   52.179622  4.644083e-242
2          Num_Bedrooms  0.056994   39.098976  2.247565e-177
3         Num_Bathrooms  0.026746   18.470720   3.123975e-62
6           Garage_Size  0.016866   11.572958   1.937848e-28
7  Neighborhood_Quality  0.001514    1.045980   2.959354e-01
0                 const  0.000000    0.000000   1.000000e+00

# Part 3 — Ridge sweep (λ = logspace(1, 1000), 10 points)
# Part 3 — Ridge sweep ($\lambda = \text{logspace}(1, 1000)$, 10 points)

In this part, we study **Ridge regression** and how its performance changes as the regularization strength varies. Ridge regression introduces an $\ell_2$ penalty that discourages large coefficients, which helps reduce overfitting and improves stability when features are correlated.

---

## Ridge regression objective

Ridge regression learns $\beta$ by minimizing the following objective:

$$
\min_{\beta}\; \|y - X\beta\|_2^2 + \lambda \|\beta\|_2^2
$$

where the $\ell_2$ penalty is:

$$
\|\beta\|_2^2 = \sum_{j=1}^{d+1} \beta_j^2
$$

- When $\lambda$ is small, Ridge behaves close to Linear Regression (weak regularization).
- When $\lambda$ is large, coefficients shrink toward zero (strong regularization), which can reduce variance but may increase bias.

---

## Sweep setup

We perform a logarithmic sweep over:

$$
\lambda \in \{1,\ldots,1000\}
$$

using **10 log-spaced values**:

$$
\lambda = \text{logspace}(0, 3, 10)
$$

This covers a broad range of weak-to-strong regularization efficiently.

---

## What we report

For **each $\lambda$**, we train Ridge regression on both:

- **Unscaled data** (raw $X$, raw $y$)
- **Scaled data** (scaled $X$ and scaled $y$)

For every trained model, we compute evaluation metrics on both the **training set** and **test set**:

- **MSE**
- **MAE**
- **$R^2$**

All metric values are reported in the **original scale of $y$**, so results from the scaled and unscaled pipelines remain directly comparable.

---

## Selecting the best Ridge model

After training all 10 models, we:

1. **Sort the results from best → worst** based on **lowest Test MSE**
2. Select


In [13]:
lambda_grid = np.logspace(-2, 2, 10)

In [14]:
def sweep_model(model_type: str, variant: str, lambda_grid: np.ndarray):
    rows = []
    for lam in lambda_grid:
        if model_type == "Ridge":
            model = Ridge(alpha=lam, random_state=42)
        elif model_type == "Lasso":
            model = Lasso(alpha=lam, max_iter=50000, random_state=42)
        elif model_type == "ElasticNet":
            model = ElasticNet(alpha=lam, l1_ratio=0.5, max_iter=50000, random_state=42)
        else:
            raise ValueError("Unknown model type")

        if variant == "unscaled":
            Xt_tr, Xt_te = X_train_un, X_test_un
            yt_tr = y_train_un
        else:
            Xt_tr, Xt_te = X_train_sc, X_test_sc
            yt_tr = y_train_sc

        model.fit(Xt_tr, yt_tr.ravel())
        pred_tr = model.predict(Xt_tr).reshape(-1,1)
        pred_te = model.predict(Xt_te).reshape(-1,1)

        # Evaluate in original y-units
        if variant == "scaled":
            pred_tr_eval = y_scaler.inverse_transform(pred_tr)
            pred_te_eval = y_scaler.inverse_transform(pred_te)
        else:
            pred_tr_eval = pred_tr
            pred_te_eval = pred_te

        mtr = eval_metrics(y_train_un, pred_tr_eval)
        mte = eval_metrics(y_test_un, pred_te_eval)

        rows.append({
            "variant": variant,
            "lambda": float(lam),
            **{f"train_{k}": v for k,v in mtr.items()},
            **{f"test_{k}": v for k,v in mte.items()},
        })
    return pd.DataFrame(rows).sort_values("test_mse").reset_index(drop=True)

def standardized_coeff_table(model, variant: str):
    beta = model.coef_.reshape(-1)
    if variant == "unscaled":
        beta_std = standardized_coef_unscaled(beta, X_train_un, y_train_un)
        return pd.DataFrame({
            "feature": feature_names,
            "coef_value": beta,
            "standardized_coef": beta_std
        }).sort_values("standardized_coef", key=np.abs, ascending=False)
    else:
        beta_orig, _ = convert_scaled_coef_to_original(beta, float(model.intercept_), x_scaler, y_scaler)
        return pd.DataFrame({
            "feature": feature_names,
            "coef_value_(scaled_space)=standardized": beta,
            "coef_value_(original_units)": beta_orig
        }).sort_values("coef_value_(scaled_space)=standardized", key=np.abs, ascending=False)

# --- Ridge ---
ridge_un = sweep_model("Ridge", "unscaled", lambda_grid)
ridge_sc = sweep_model("Ridge", "scaled", lambda_grid)

print("Ridge — Unscaled (best → worst):")
display(ridge_un)

print("\nRidge — Scaled (best → worst):")
display(ridge_sc)

best_lam_un = ridge_un.iloc[0]["lambda"]
best_lam_sc = ridge_sc.iloc[0]["lambda"]

ridge_best_un = Ridge(alpha=best_lam_un, random_state=42).fit(X_train_un, y_train_un.ravel())
ridge_best_sc = Ridge(alpha=best_lam_sc, random_state=42).fit(X_train_sc, y_train_sc.ravel())

print("\nBest Ridge (UNSCALED) standardized coefficients:")
display(standardized_coeff_table(ridge_best_un, "unscaled"))

print("\nBest Ridge (SCALED) standardized coefficients:")
display(standardized_coeff_table(ridge_best_sc, "scaled"))


Ridge — Unscaled (best → worst):


variant      lambda     train_mse    train_mae  train_r2      test_mse     test_mae   test_r2
0  unscaled    4.641589  9.278479e+07  7617.688720  0.998559  1.022728e+08  8093.072477  0.998398
1  unscaled    1.668101  9.277305e+07  7615.475486  0.998559  1.022748e+08  8090.757899  0.998398
2  unscaled    0.599484  9.277152e+07  7614.877777  0.998559  1.022784e+08  8090.002437  0.998398
3  unscaled    0.215443  9.277132e+07  7614.690695  0.998559  1.022800e+08  8089.730526  0.998398
4  unscaled    0.077426  9.277130e+07  7614.623421  0.998559  1.022807e+08  8089.632754  0.998398
5  unscaled    0.027826  9.277129e+07  7614.599238  0.998559  1.022809e+08  8089.597609  0.998398
6  unscaled    0.010000  9.277129e+07  7614.590547  0.998559  1.022810e+08  8089.584978  0.998398
7  unscaled   12.915497  9.287366e+07  7626.168139  0.998558  1.023278e+08  8104.212477  0.998397
8  unscaled   35.938137  9.352048e+07  7668.674668  0.998548  1.029106e+08  8153.785890  0.998388
9  unscaled  100.000000  9.779134e+07  7897.836421  0.998481  1.071971e+08  8369.765680  0.998321


Ridge — Scaled (best → worst):


variant      lambda     train_mse     train_mae  train_r2      test_mse      test_mae   test_r2
0  scaled    0.010000  9.277131e+07   7614.549562  0.998559  1.022927e+08   8090.285778  0.998398
1  scaled    0.027826  9.277139e+07   7614.485201  0.998559  1.023135e+08   8091.547578  0.998397
2  scaled    0.077426  9.277208e+07   7614.306140  0.998559  1.023718e+08   8095.058285  0.998397
3  scaled    0.215443  9.277736e+07   7613.949635  0.998559  1.025373e+08   8104.824497  0.998394
4  scaled    0.599484  9.281820e+07   7613.976029  0.998559  1.030241e+08   8132.380363  0.998386
5  scaled    1.668101  9.313331e+07   7621.465972  0.998554  1.045802e+08   8211.917753  0.998362
6  scaled    4.641589  9.554996e+07   7714.472602  0.998516  1.104499e+08   8472.424225  0.998270
7  scaled   12.915497  1.137747e+08   8397.410208  0.998233  1.382717e+08   9513.677047  0.997834
8  scaled   35.938137  2.451149e+08  12827.064289  0.996194  2.960397e+08  14209.058577  0.995363
9  scaled  100.000000  1.087270e+09  27890.863380  0.983116  1.206850e+09  29685.163263  0.981097


Best Ridge (UNSCALED) standardized coefficients:


/tmp/ipython-input-3445319319.py:15: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  std_y = float(np.std(y_train_raw, axis=0, ddof=0))


feature    coef_value  standardized_coef
0        Square_Footage    199.359341           0.984782
3            Year_Built   1002.441326           0.081828
4              Lot_Size  14785.992351           0.075665
1          Num_Bedrooms  10207.572238           0.056825
2         Num_Bathrooms   8235.421429           0.026499
5           Garage_Size   5159.156546           0.016726
6  Neighborhood_Quality    133.842071           0.001532


Best Ridge (SCALED) standardized coefficients:


/tmp/ipython-input-3445319319.py:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_mean = float(y_scaler.mean_)
/tmp/ipython-input-3445319319.py:24: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_std  = float(y_scaler.scale_)


feature  coef_value_(scaled_space)=standardized  coef_value_(original_units)
0        Square_Footage                                0.984739                   199.350781
3            Year_Built                                0.081852                  1002.732899
4              Lot_Size                                0.075967                 14845.024686
1          Num_Bedrooms                                0.056993                 10237.761326
2         Num_Bathrooms                                0.026745                  8311.878916
5           Garage_Size                                0.016866                  5202.414632
6  Neighborhood_Quality                                0.001514                   132.216520

# Part 4 — Lasso sweep ($\lambda = \text{logspace}(1, 1000)$, 10 points)

In this part, we analyze **Lasso regression** and investigate how increasing the regularization strength affects:
- predictive performance (generalization error), and  
- the learned coefficients (sparsity / feature selection).

Unlike Ridge (which shrinks coefficients smoothly), Lasso tends to push many coefficients **exactly to zero**, making it a useful method for **automatic feature selection**.

---

## Lasso regression objective

Lasso regression learns $\beta$ by minimizing:

$$
\min_{\beta}\; \|y - X\beta\|_2^2 + \lambda \|\beta\|_1
$$

where the $\ell_1$ penalty is:

$$
\|\beta\|_1 = \sum_{j=1}^{d+1} |\beta_j|
$$

Key behaviors:
- Small $\lambda$  $\Rightarrow$ close to Linear Regression (weak regularization)  
- Large $\lambda$ $\Rightarrow$ many coefficients become exactly **0** (strong regularization)

---

## Sweep setup

We perform a logarithmic sweep over:

$$
\lambda \in \{1,\ldots,1000\}
$$

using **10 log-spaced values**:

$$
\lambda = \text{logspace}(0, 3, 10)
$$

This allows us to test a wide range of regularization strengths efficiently.

---

## What we report

For each $\lambda$, we train Lasso regression on both:

- **Unscaled data** (raw $X$, raw $y$)
- **Scaled data** (scaled $X$ and scaled $y$)

For every trained model, we compute evaluation metrics on both the **training set** and **test set**:

- **MSE**
- **MAE**
- **$R^2$**

All metrics are reported in the **original scale of $y$**, meaning results are directly comparable between scaled and unscaled experiments.

---

## Selecting the best Lasso model

After evaluating all 10 $\lambda$ values, we:

1. **Sort the results from best → worst** based on **lowest Test MSE**
2. Select the best-performing $\lambda$
3. Report the **standardized coefficients** of the best Lasso model for both:
   - Unscaled pipeline (computed using feature and output standard deviations)
   - Scaled pipeline (coefficients are already standardized effects since both $X$ and $y$ are scaled)

✅ **Goal of Part 4:**  
To study how the $\ell_1$ penalty trades off predictive accuracy vs. sparsity, and to identify the best-performing Lasso model based on test performance.


In [15]:
# --- Lasso ---
lasso_un = sweep_model("Lasso", "unscaled", lambda_grid)
lasso_sc = sweep_model("Lasso", "scaled", lambda_grid)

print("Lasso — Unscaled (best → worst):")
display(lasso_un)

print("\nLasso — Scaled (best → worst):")
display(lasso_sc)

best_lam_un = lasso_un.iloc[0]["lambda"]
best_lam_sc = lasso_sc.iloc[0]["lambda"]

lasso_best_un = Lasso(alpha=best_lam_un, max_iter=50000, random_state=42).fit(X_train_un, y_train_un.ravel())
lasso_best_sc = Lasso(alpha=best_lam_sc, max_iter=50000, random_state=42).fit(X_train_sc, y_train_sc.ravel())

print("\nBest Lasso (UNSCALED) standardized coefficients:")
display(standardized_coeff_table(lasso_best_un, "unscaled"))

print("\nBest Lasso (SCALED) standardized coefficients:")
display(standardized_coeff_table(lasso_best_sc, "scaled"))


Lasso — Unscaled (best → worst):


variant      lambda     train_mse    train_mae  train_r2      test_mse     test_mae   test_r2
0  unscaled  100.000000  9.280972e+07  7620.617426  0.998559  1.022018e+08  8089.656461  0.998399
1  unscaled   35.938137  9.277626e+07  7616.229543  0.998559  1.022440e+08  8089.334790  0.998399
2  unscaled   12.915497  9.277193e+07  7615.115392  0.998559  1.022666e+08  8089.487605  0.998398
3  unscaled    4.641589  9.277138e+07  7614.771446  0.998559  1.022757e+08  8089.545548  0.998398
4  unscaled    1.668101  9.277130e+07  7614.652427  0.998559  1.022791e+08  8089.566372  0.998398
5  unscaled    0.599484  9.277129e+07  7614.609666  0.998559  1.022804e+08  8089.573696  0.998398
6  unscaled    0.215443  9.277129e+07  7614.594295  0.998559  1.022808e+08  8089.576385  0.998398
7  unscaled    0.077426  9.277129e+07  7614.588770  0.998559  1.022810e+08  8089.577352  0.998398
8  unscaled    0.027826  9.277129e+07  7614.586785  0.998559  1.022810e+08  8089.577699  0.998398
9  unscaled    0.010000  9.277129e+07  7614.586071  0.998559  1.022810e+08  8089.577822  0.998398


Lasso — Scaled (best → worst):


variant      lambda     train_mse      train_mae  train_r2      test_mse       test_mae   test_r2
0  scaled    0.010000  1.299294e+08    9108.827138  0.997982  1.387411e+08    9682.054933  0.997827
1  scaled    0.027826  3.595940e+08   15231.221110  0.994416  3.580094e+08   15497.598853  0.994392
2  scaled    0.077426  1.477932e+09   31136.942917  0.977050  1.365326e+09   29729.172852  0.978615
3  scaled    0.215443  4.121166e+09   53064.964770  0.936004  3.887817e+09   51506.135047  0.939105
4  scaled    0.599484  2.427527e+10  132947.942022  0.623038  2.381652e+10  132238.305447  0.626961
5  scaled    1.668101  6.439721e+10  217690.134646  0.000000  6.384958e+10  218131.524136 -0.000079
6  scaled    4.641589  6.439721e+10  217690.134646  0.000000  6.384958e+10  218131.524136 -0.000079
7  scaled   12.915497  6.439721e+10  217690.134646  0.000000  6.384958e+10  218131.524136 -0.000079
8  scaled   35.938137  6.439721e+10  217690.134646  0.000000  6.384958e+10  218131.524136 -0.000079
9  scaled  100.000000  6.439721e+10  217690.134646  0.000000  6.384958e+10  218131.524136 -0.000079


Best Lasso (UNSCALED) standardized coefficients:


/tmp/ipython-input-3445319319.py:15: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  std_y = float(np.std(y_train_raw, axis=0, ddof=0))


feature    coef_value  standardized_coef
0        Square_Footage    199.360105           0.984785
3            Year_Built   1001.991254           0.081792
4              Lot_Size  14785.749535           0.075663
1          Num_Bedrooms  10198.054386           0.056772
2         Num_Bathrooms   8172.740838           0.026297
5           Garage_Size   5066.149429           0.016424
6  Neighborhood_Quality    123.375148           0.001413


Best Lasso (SCALED) standardized coefficients:


/tmp/ipython-input-3445319319.py:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_mean = float(y_scaler.mean_)
/tmp/ipython-input-3445319319.py:24: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_std  = float(y_scaler.scale_)


feature  coef_value_(scaled_space)=standardized  coef_value_(original_units)
0        Square_Footage                                0.975483                   197.476874
3            Year_Built                                0.070753                   866.759491
4              Lot_Size                                0.066821                 13057.793236
1          Num_Bedrooms                                0.047341                  8503.998153
2         Num_Bathrooms                                0.017079                  5307.916533
5           Garage_Size                                0.008307                  2562.327817
6  Neighborhood_Quality                                0.000000                     0.000000

# Part 5 — ElasticNet sweep ($\lambda = \text{logspace}(1, 1000)$, 10 points)

In this part, we investigate **Elastic-Net regression**, which combines the strengths of:
- **Ridge regression** (stable shrinkage under multicollinearity), and
- **Lasso regression** (sparse feature selection).

Elastic-Net is often preferred when features are correlated, because pure Lasso may behave inconsistently by selecting one feature from a correlated group, while Elastic-Net can distribute weight more smoothly.

---

## Elastic-Net regression objective

Elastic-Net learns $\beta$ by minimizing:

$$
\min_{\beta}\; \|y - X\beta\|_2^2
+ \lambda\Big( r\|\beta\|_1 + (1-r)\|\beta\|_2^2 \Big)
$$

where:
- $\|\beta\|_1 = \sum_{j=1}^{d+1} |\beta_j|$ is the $\ell_1$ penalty,
- $\|\beta\|_2^2 = \sum_{j=1}^{d+1} \beta_j^2$ is the $\ell_2$ penalty,
- $\lambda$ controls the overall regularization strength,
- $r$ controls the mixing between Lasso and Ridge behavior.

In this notebook, we use:

$$
r = 0.5
$$

which gives equal importance to $\ell_1$ and $\ell_2$ penalties.

---

## Sweep setup

We perform a logarithmic sweep over:

$$
\lambda \in \{1,\ldots,1000\}
$$

using **10 log-spaced values**:

$$
\lambda = \text{logspace}(0, 3, 10)
$$

This provides coverage from weak to strong regularization.

---

## What we report

For each $\lambda$, Elastic-Net is trained on both:

- **Unscaled data** (raw $X$, raw $y$)
- **Scaled data** (scaled $X$ and scaled $y$)

For every trained model, we compute evaluation metrics on both the **training set** and **test set**:

- **MSE**
- **MAE**
- **$R^2$**

All metrics are reported in the **original scale of $y$**, so results from scaled and unscaled pipelines are directly comparable.

---

## Selecting the best Elastic-Net model

After evaluating all 10 $\lambda$ values, we:

1. **Sort the results from best → worst** based on **lowest Test MSE**
2. Select the best-performing $\lambda$
3. Report the **standardized coefficients** of the best Elastic-Net model for both:
   - Unscaled pipeline (computed using feature and output standard deviations)
   - Scaled pipeline (coefficients are already standardized effects since both $X$ and $y$ are scaled)

✅ **Goal of Part 5:**  
To understand how a mixed $\ell_1+\ell_2$ penalty balances sparsity and stability, and to identify the best-performing Elastic-Net model based on test performance.


In [16]:
# --- ElasticNet ---
enet_un = sweep_model("ElasticNet", "unscaled", lambda_grid)
enet_sc = sweep_model("ElasticNet", "scaled", lambda_grid)

print("ElasticNet — Unscaled (best → worst):")
display(enet_un)

print("\nElasticNet — Scaled (best → worst):")
display(enet_sc)

best_lam_un = enet_un.iloc[0]["lambda"]
best_lam_sc = enet_sc.iloc[0]["lambda"]

enet_best_un = ElasticNet(alpha=best_lam_un, l1_ratio=0.5, max_iter=50000, random_state=42).fit(X_train_un, y_train_un.ravel())
enet_best_sc = ElasticNet(alpha=best_lam_sc, l1_ratio=0.5, max_iter=50000, random_state=42).fit(X_train_sc, y_train_sc.ravel())

print("\nBest ElasticNet (UNSCALED) standardized coefficients:")
display(standardized_coeff_table(enet_best_un, "unscaled"))

print("\nBest ElasticNet (SCALED) standardized coefficients:")
display(standardized_coeff_table(enet_best_sc, "scaled"))


ElasticNet — Unscaled (best → worst):


variant      lambda     train_mse     train_mae  train_r2      test_mse      test_mae   test_r2
0  unscaled    0.010000  9.277899e+07   7616.743145  0.998559  1.022722e+08   8092.154037  0.998398
1  unscaled    0.027826  9.282998e+07   7622.608061  0.998558  1.022964e+08   8098.741131  0.998398
2  unscaled    0.077426  9.320655e+07   7648.825274  0.998553  1.026162e+08   8133.250009  0.998393
3  unscaled    0.215443  9.578327e+07   7797.562304  0.998513  1.051532e+08   8275.715956  0.998353
4  unscaled    0.599484  1.105867e+08   8438.988439  0.998283  1.204786e+08   8884.238124  0.998113
5  unscaled    1.668101  1.714670e+08  10491.630041  0.997337  1.847600e+08  10907.246650  0.997106
6  unscaled    4.641589  3.197836e+08  14413.648266  0.995034  3.425893e+08  14830.987829  0.994634
7  unscaled   12.915497  5.086138e+08  18272.098166  0.992102  5.435766e+08  18771.604761  0.991486
8  unscaled   35.938137  6.429061e+08  20560.049187  0.990017  6.846735e+08  21172.158543  0.989276
9  unscaled  100.000000  7.115846e+08  21614.689259  0.988950  7.518187e+08  22235.815431  0.988224


ElasticNet — Scaled (best → worst):


variant      lambda     train_mse      train_mae  train_r2      test_mse       test_mae   test_r2
0  scaled    0.010000  1.073885e+08    8243.899428  0.998332  1.204503e+08    8958.371723  0.998113
1  scaled    0.027826  2.030448e+08   11418.996076  0.996847  2.169898e+08   12137.090857  0.996601
2  scaled    0.077426  8.126667e+08   23081.596491  0.987380  8.015032e+08   23132.383551  0.987446
3  scaled    0.215443  3.546624e+09   48985.146673  0.944926  3.330184e+09   47372.583749  0.947839
4  scaled    0.599484  1.471103e+10  103026.831082  0.771558  1.431875e+10  101810.394835  0.775725
5  scaled    1.668101  5.393359e+10  199077.624327  0.162486  5.339324e+10  199283.369868  0.163699
6  scaled    4.641589  6.439721e+10  217690.134646  0.000000  6.384958e+10  218131.524136 -0.000079
7  scaled   12.915497  6.439721e+10  217690.134646  0.000000  6.384958e+10  218131.524136 -0.000079
8  scaled   35.938137  6.439721e+10  217690.134646  0.000000  6.384958e+10  218131.524136 -0.000079
9  scaled  100.000000  6.439721e+10  217690.134646  0.000000  6.384958e+10  218131.524136 -0.000079


Best ElasticNet (UNSCALED) standardized coefficients:


/tmp/ipython-input-3445319319.py:15: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  std_y = float(np.std(y_train_raw, axis=0, ddof=0))


feature    coef_value  standardized_coef
0        Square_Footage    199.357951           0.984775
3            Year_Built   1002.516354           0.081834
4              Lot_Size  14800.436183           0.075739
1          Num_Bedrooms  10215.051326           0.056866
2         Num_Bathrooms   8254.146826           0.026559
5           Garage_Size   5169.670494           0.016760
6  Neighborhood_Quality    133.450298           0.001528


Best ElasticNet (SCALED) standardized coefficients:


/tmp/ipython-input-3445319319.py:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_mean = float(y_scaler.mean_)
/tmp/ipython-input-3445319319.py:24: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_std  = float(y_scaler.scale_)


feature  coef_value_(scaled_space)=standardized  coef_value_(original_units)
0        Square_Footage                                0.975156                   197.410685
3            Year_Built                                0.075907                   929.902941
4              Lot_Size                                0.071650                 14001.514974
1          Num_Bedrooms                                0.051628                  9274.043694
2         Num_Bathrooms                                0.021679                  6737.402607
5           Garage_Size                                0.012833                  3958.237650
6  Neighborhood_Quality                                0.000000                     0.000000